In [1]:
import psi4
import numpy as np
import sys
sys.path.append('../lib')
from P4toC4_aux import *

In [7]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "sto-3g")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "RHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

# O
# H 1 0.96
# H 1 0.96 2 104.5

h2o = psi4.geometry("""
0 1
O            0.000000000000     0.000000000000    -0.065775570547
H            0.000000000000    -0.759061990794     0.521953018286
H            0.000000000000     0.759061990794     0.521953018286
no_reorient
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-74.96331905260071

In [15]:
# Print a new basis set in GENBAS format
basisset=wf.basisset()
#print(basisset.genbas())

In [23]:
mapping, scale = basis_mapping(wf.basisset(), verbose=0)
mapping

array([ 0,  1,  2,  7,  3,  5,  8,  4,  6,  9, 11, 13, 12, 10, 14, 15, 18,
       16, 17, 19, 20, 23, 21, 22])

In [12]:
psi4.core.Vector.array_interface(wf.epsilon_a())

[array([-20.24209878,  -1.26699811,  -0.45270347,   0.60291843]),
 array([], dtype=float64),
 array([-0.39107408]),
 array([-0.61642276,  0.73901693])]

In [5]:
n_a, n_b = wf.nalpha(), wf.nbeta()
print(f'Na={n_a}  Nb={n_b}')
eps_a = np.array(wf.epsilon_a())
eps_b = np.array(wf.epsilon_a())
s=27.2114

for i in range(0,max(n_a,n_b)+2):
    print(f' {i+1:3d}  {eps_a[i]*s:10.4f} {eps_b[i]*s:10.4f}')

Na=5  Nb=5


/home/thomas/anaconda3/envs/psi4/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'psi4.core.Vector'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  This is separate from the ipykernel package so we can avoid doing imports until
/home/thomas/anaconda3/envs/psi4/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'psi4.core.Vector'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  after removing the cwd from sys.path.


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [22]:
# Psi4
Ca=np.array(wf.Ca())
Cb=np.array(wf.Cb())

In [24]:
Ca_C4 = psi4_to_c4(Ca, mapping, scale)
Cb_C4 = psi4_to_c4(Cb, mapping, scale)

In [25]:
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)